In [43]:
import math
from termcolor import colored
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, scale
import joblib
from keras.models import Sequential
from keras.layers import Dense, LSTM, Masking
from keras.layers import Dropout
from keras.models import load_model
import matplotlib.pyplot as plt
from prepare_data import get_data
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error

In [44]:
def get_dataset(train_data):
    """ Fills len(train_data) long arrays with 60 records 
        in each position
        example = [[1...60], [1...60], ...]
    """
    x_train = []
    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, :-1])
    x_train = np.array(x_train)
    return np.array(x_train)

In [45]:
def create_model(shape_examples, shape_features):
    print(colored('Generating LSTM model.', 'yellow'))
    print(colored(f'Shape {shape_examples}x{shape_features}', 'cyan'))

    model = Sequential()
    model.add(LSTM(
        units=50,
        return_sequences=True,
        input_shape=(shape_examples, shape_features)
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(units=50))
    model.add(Dropout(0.25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=["accuracy"])
    return model


In [46]:
try:
    X = np.load('resources/X_3.npy')
    Y = np.load('resources/Y_3.npy')
    scaler = joblib.load("resources/scaler_3.save")
    print(colored('Loaded storaged data.', 'green'))
except FileNotFoundError:
    try:
        df = pd.read_csv("resources/test.csv")
    except FileNotFoundError:
        print(colored('Creating missing file.', 'red'))
        get_data()
        df = pd.read_csv("resources/test.csv")

    df["trend"] = df["AO_1_500"].shift(-1)
    df = df.iloc[:, 2:]
    print(colored('Setting up parameters.', 'yellow'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df.values)  # shape (70718, 8)
    joblib.dump(scaler, "resources/scaler_3.save")
    X = get_dataset(scaled_data)
    Y = np.array(scaled_data[60:len(scaled_data), -1])
    np.save("resources/X_3.npy", X)
    np.save("resources/Y_3.npy", Y)


Loaded storaged data.


In [47]:
print(colored(f'X shape: {X.shape}', 'cyan'))
print(colored(f'Y shape: {Y.shape}', 'cyan'))

training_data_len = math.ceil(len(Y) * .8)
x_train = X[:training_data_len]
y_train = Y[:training_data_len]
x_train = np.array(x_train)
y_train = np.array(y_train)

print(colored(f'X_train shape: {x_train.shape}', 'yellow'))
print(colored(f'Y_train shape: {y_train.shape}', 'yellow'))

X shape: (70151, 60, 7)
Y shape: (70151,)
X_train shape: (56121, 60, 7)
Y_train shape: (56121,)


In [48]:
try:
    model = load_model('resources/my_model_3.h5')
    print(colored('Model loaded successfully', 'green'))
except IOError:
    print(colored('Training...', 'yellow'))
    model = create_model(X.shape[1], X.shape[-1])
    history = model.fit(x_train, y_train, batch_size=64, epochs=10)
    model.save('resources/my_model_3.h5')
    hist_df = pd.DataFrame(history.history)
    with open("resources/history.csv", mode='w') as f:
        hist_df.to_csv(f)


Model loaded successfully


In [49]:
train_acc = model.evaluate(x_train, y_train, verbose=0)
train_acc

[0.00014471950998995453, 0.0]

In [63]:
index = 6
sample = np.array([x_train[index]])

y_sample = scaler.inverse_transform([[0] * 7 + [y_train[index]]])
print(y_sample[0][-1])
prediction = model.predict(sample)[0][0]
data = scaler.inverse_transform([[0] * 7 + [prediction]])
print(data[0][-1])
# print("train: ", mean_squared_error(y_train, prediction))


406.25550000000015
1/1 [==============================] - 0s 49ms/step
340.4544993026811
